# 데이터 가공
경기 데이터를 분석하기 용이하게끔 약간의 수정 및 처리를 가하여줍니다.

## 라이브러리 불러오기
- 자료 분석을 위하여 numpy, pandas 라이브러리를 사용합니다.
- json 파일을 읽어오기 위하여 json 라이브러리를 사용합니다.

In [1]:
import numpy as np
import pandas as pd
import json

## 경기 데이터 가공하기
경기 데이터를 읽어와 필요 없는 부분은 자르고 양이 많은 부분은 따로 저장해줍니다.

In [2]:
with open("data/data_summoner_match_data_야식은치킨이지.json", 'r') as file:
    match_json_data = json.load(file)

# 원 데이터를 평면화하여 데이터프레임으로 저장합니다. (원 데이터는 match_json_data에 남아 있습니다.)
df = pd.json_normalize(match_json_data)

# 경기의 meta 데이터 중 matchId와 participants(puuid)의 정보를 담은 df_meta라는 변수를 만들어줍니다.
df_meta = df.iloc[:, 1:3]
df_meta.columns = ['matchId', 'participants']

# 남은 경기 데이터 정보를 담은 df_info라는 변수를 만들어줍니다.
df_info = df.iloc[:, 3:]
df_info.columns = match_json_data[0]['info'].keys()

# 가져온 경기의 총 횟수를 따로 저장해줍니다.
match_length = len(match_json_data)

# df_info에 담긴 데이터 중 상당한 부분을 차지하는 teams 변수 부분을 따로 빼내어 저장해줍니다.
arr_teams = df_info.pop('teams')
df_teams = pd.concat([pd.json_normalize(arr_teams[x]) for x in range(match_length)])
df_teams.index = range(match_length*2)

# df_info에 담긴 데이터 중 상당한 부분을 차지하는 participants 변수 부분을 따로 빼내어 저장해줍니다.
arr_participants = df_info.pop('participants')
df_participants = pd.concat([pd.json_normalize(arr_participants[x]) for x in range(match_length)])
df_participants.index = range(match_length*10)

## 분석하고자 하는 소환사의 puuid 알아내기

In [3]:
# summoner 데이터 파일에는 소환사의 id, accountId, puuid 등의 값들이 저장되어있습니다.
with open("data/data_summoner_야식은치킨이지.json", 'r') as file:
    summoner_puuid = json.load(file)["puuid"]    
summoner_puuid

'rloB_NxGn8Lyi7tAdxrzRxlN9F7ohuDouptz8KV3Mlk2Epd-qnckbDLlIAnX7ABWVt4SG42M2ca5gw'

## 경기 데이터에서 소환사의 index 알아내기
경기 데이터는 많은 양의 값이 순서가 규칙성을 띄며 섞여 있습니다. 따라서 원하는 데이터를 얻어오기 위해선 소환사의 데이터가 무엇인지 분류해낼 수 있어야 하며, 앞에서 구한 puuid를 통해 알 수 있습니다.

In [4]:
df_meta['summonerIndex'] = df_meta.participants.map(lambda participants : participants.index(summoner_puuid))
df_meta.head()

,matchId,participants,summonerIndex
0,KR_5383029593,[_-0Xyk-14U-wYGaw3wjoflpZQumQL4ucbCWNMCP7oylV0...,6
1,KR_5383081525,[fnIqdrkSIgTXVDwwF56NqJnMkdhyX_d4AncrJhsEnzptM...,5
2,KR_5382976870,[rloB_NxGn8Lyi7tAdxrzRxlN9F7ohuDouptz8KV3Mlk2E...,0
3,KR_5382817808,[r4xI-xms6dhwW33Pa0Ci4eyp8zPumTTe3uuxpsQJuggw3...,3
4,KR_5382882767,[nUEvQyRsfBZGM-ccusJ2-fnXWxnD3IqWi5jfdqHMYmnbY...,5


## 경기의 시간별 데이터 가져오기 (timeline)
### 데이터의 흐름
- match_length 길이의 타임라인 데이터
    - metadata 
        - dataVersion
        - matchId
        - participants (puuid)
    - info
        - frameInterval
        - frames
            - [시간, 0 ~ 게임 종료 시간(frameInterval이 60 * 1000 이니까 분 단위라고 추측할 수 있음)]
                - event (발생한 이벤트에 대한 정보)
                    - {공통된 정보}
                        - participantsId
                        - timestamp
                        - type
                    - {type에 따라 주어지는 추가적인 정보}
                        - itemId
                        - killerId
                        - position
                        - ...
                - participantFrames (현 시점에 경기 참가자에 대한 정보)
                    - championStats = { ... }
                    - damageStats = { ... }
                    - currentGold
                    - position
                    - minionKilled
                    - ...

In [5]:
with open("data/data_summoner_match_timeline_data_야식은치킨이지.json", "r") as file:
    json_data = json.load(file)

# 타임라인 데이터는 굉장히 복잡하기때문에 평면화를 바로 시켜주는 대신 객체를 먼저 meta 데이터와 info 데이터로 나누어줍니다.
tl_meta = [match["metadata"] for match in json_data]
df_tl_meta = pd.json_normalize(tl_meta)                     # 비교적 단순한 meta 데이터는 바로 데이터프레임으로 만들어줍니다.
tl_info = [match["info"]["frames"] for match in json_data]  
frame_interval = 60000                                      # maybe 60 * 1000 milliseconds


# LCK 따라잡기
## to-do
- KDA
- DPM (분당 데미지)
- DMG% (팀 내 데미지 비율)
- DMG/GOLD (골드 당 데미지)
- CSD@15 (15분 CS 차이)
- GD@15 (15분 골드 차이)
- JP-D (정글러 관여율 차이)
+ DMG@END, DMG@15
+ 주라인에 따른 위의 통계량


## KDA 구하기

In [6]:
# data 변수는 그 때 그 때 사용하는 데이터를 담는 변수입니다.
data = df_participants[df_participants.puuid == summoner_puuid].reset_index()   # 경기 참가자 중 소환사의 데이터를 찾아 따로 저장합니다.
display(data[["kills", "deaths", "assists"]].describe().iloc[1:])               # count index는 제외합니다.

# kda 구하기
kda = (data["kills"]+data["assists"])/data["deaths"]
kda.name = "kda"
kda[data["deaths"]==0] = -1                                                     # perfect score = -1

# 구한 데이터는 따로 저장하여 이후 활용하도록 하겠습니다.
data_kda = pd.concat([data[["win", "kills", "deaths", "assists"]], kda], axis=1)
display(data_kda.groupby(['win']).apply(lambda df: df.mean()))

,kills,deaths,assists
mean,5.180000,6.310000,12.510000
std,3.870061,3.526795,9.564766
min,0.000000,0.000000,0.000000
25%,2.750000,3.000000,4.000000
50%,4.000000,7.000000,9.500000
75%,7.000000,9.000000,20.000000
max,23.000000,14.000000,33.000000


,win,kills,deaths,assists,kda
win,,,,,
False,0.0,4.468085,7.446809,8.723404,1.875234
True,1.0,5.811321,5.301887,15.867925,4.989877


## DPM 구하기

In [7]:
### DPM
minute_played = data.timePlayed/60      # data.timePlayed 의 단위는 '초'입니다.

# 포탑, 미니언, 중립몬스터, 챔피언 등 모두에게 넣은 데미지와 챔피언에게만 넣은 데미지는 구분되어있습니다.
dpm_all = (data.totalDamageDealt/minute_played)
dpm_to_champions = data.totalDamageDealtToChampions/minute_played

dpm = pd.concat([dpm_all, dpm_to_champions], axis=1, keys=["dpm_all", "dpm_to_champions"])
display(dpm.head())

data_dpm = dpm
# additional|
# display(data.totalTimeSpentDead.describe()/60)
# display(data_kda.describe())
# display(data.totalMinionsKilled.describe())

,dpm_all,dpm_to_champions
0,2541.532004,843.525708
1,3749.178295,591.689922
2,2797.524116,573.231511
3,5493.585237,620.272408
4,3980.109170,690.327511


## DMG% 구하기

In [8]:
list_dmg_per = []
for i in range(match_length):
    dmg_team = 0
    dmg_me = 0
    for j in range(10):
        if (df_participants.iloc[i*10 + j].puuid == summoner_puuid):
            dmg_me = dmg_me + df_participants.iloc[i*10 + j].totalDamageDealtToChampions
        dmg_team = dmg_team + df_participants.iloc[i*10 + j].totalDamageDealtToChampions
    dmg_per = dmg_me/dmg_team
    list_dmg_per.append(dmg_per)

data_dmg_per = pd.Series(list_dmg_per, name="dmg_percent")
data_dmg_per.head()

0    0.098180
1    0.101546
2    0.101423
3    0.093030
4    0.192651
Name: dmg_percent, dtype: float64

## DMG/GOLD 구하기

In [9]:
dmg = data.totalDamageDealtToChampions
gold = data.goldEarned

data_dmg_by_gold = dmg/gold
data_dmg_by_gold.name = "dmg_by_gold"
data_dmg_by_gold.head()

0    1.301029
1    1.580421
2    1.593164
3    1.479904
4    1.852197
Name: dmg_by_gold, dtype: float64

## 포지션 정보 구하기
- 라인을 알 수 없는 경우(Invalid)는 주로 칼바람이나 넥서스 파괴 모드 등 특별한 모드입니다
- 언제나 lane 데이터와 individualPosition 데이터가 일치하는 것은 아닙니다
    - individualPosition이 더 괜찮은 지표라고 판단하여 아래에서 사용하였습니다.

In [10]:
data_position = pd.concat([data.lane, data.individualPosition], axis=1)
data_position.head()

,lane,individualPosition
0,NONE,Invalid
1,TOP,TOP
2,BOTTOM,BOTTOM
3,BOTTOM,BOTTOM
4,NONE,TOP


## 맞라인 상대의 정보 구하기
- CSD@15, GD@15, JP-D 같은 경우 팀 전체의 데이터보다 맞 상대와의 차이를 통해 구하는 것이 더 유의미하다고 판단하였습니다.
    - e.g. 서포터의 골드 vs 황족 미드의 골드 = (무의미한 값)
- 여기서부턴 dataframe 간의 연산이 아닌, 첫 번째 경기부터 차례차례 연산을 진행하였습니다.
    - 처음은 하나의 경기를 고정하고 통계치를 구하고, 모든 값을 정상적으로 구했을 때 모든 경기에 대한 통계치를 구합니다.

### 맞라인 상대와 각 팀 정글러의 index 구하기

In [11]:
match_index = 1     # 임시, 두 번째 경기를 분석하겠다는 의미입니다.

# data 변수엔 본인의 데이터만 저장되어 있습니다.
summoner_position = data.individualPosition[match_index]    # 소환사의 포지션을 확인합니다.
if summoner_position == "invalid": exit() # continue        # 소환사의 포지션이 invalid 즉, 다른 모드의 케이스는 무시합니다.

# 따라서 다른 소환사의 데이터를 다루기 위해 새로운 변수를 선언합니다.
participant = df_participants.iloc[match_index*10: match_index*10+10].reset_index() # 해당 경기의 소환사들의 데이터를 불러옵니다.

jungler_index = [-1, -1]    # [블루팀 정글러의 index, 레드팀 정글러의 index]
for i in range(10):
    # 각 팀 정글러의 index 구하기
    if participant.individualPosition[i] == "JUNGLE":
        if i < 5: jungler_index[0] = i
        else: jungler_index[1] = i
    # 내 index 구하기
    if participant.puuid[i] == summoner_puuid: 
        summoner_index = i
        if summoner_index < 5 : summoner_team = 0   # 0: blue / 1: red
        else: summoner_team = 1                     # 본인 팀에 대한 지표로, 정글러의 index를 불러올 때 사용합니다. 
    # 맞라인 상대의 index 구하기
    elif participant.individualPosition[i] == summoner_position:
        rival = participant.iloc[i]
        rival_index = i

## CSD@15 (15분 CS 차이) 구하기

In [12]:
# 'match_index' 번째 경기의 tl_info로부터 't 분' 시점의 '참가자들의 데이터'를 가져옵니다.
tl_participants_frames          = [tl_info[match_index][t]["participantFrames"] for t in range(len(tl_info[match_index]))]
tl_15_participants_frames       = tl_participants_frames[15]  # 15분 데이터
df_tl_15_participants_frames    = pd.json_normalize([tl_15_participants_frames[str(x+1)] for x in range(10)])

csd = df_tl_15_participants_frames.minionsKilled[summoner_index] - df_tl_15_participants_frames.minionsKilled[rival_index]
print("CSD@15 = %d" % csd)

CSD@15 = 52


## GD@15 (15분 GOLD 차이) 구하기

In [13]:
gd = df_tl_15_participants_frames.totalGold[summoner_index] - df_tl_15_participants_frames.totalGold[rival_index]
print("GD@15 = %d" % gd)

GD@15 = 1558


## JP-D (정글러 관여율 차이) 구하기

### 1. 킬 이벤트 가져오기

In [14]:
# 'match_index' 번째 경기의 tl_info로부터 't 분' 시점의 '발생한 이벤트 목록'을 가져옵니다.
tl_events = [tl_info[match_index][t]["events"] for t in range(len(tl_info[match_index]))]
tl_kill_events = []
for i in range(len(tl_events)):
    for event in tl_events[i]:
        if event["type"] == "CHAMPION_KILL":
            tl_kill_events.append(pd.json_normalize(event))
            
# 배열로부터 생성된 데이터프레임은 index를 다시 설정해주는 것이 좋습니다.
df_kill_events = pd.concat(tl_kill_events).reset_index()    
del df_kill_events['index']

### 2. 정글러 정보 가져오기

In [15]:
# 주의! 타임라인 데이터에서의 참가자들의 index는 1부터 시작합니다. (앞서 다루었던 index는 전부 0~9 사이의 값)

# 앞에서 구한 "본인의 index, 본인 팀 정글의 index / 맞라인 상대의 index, 상대 팀 정글의 index"를 출력해보았습니다.
print(summoner_index+1, ",", jungler_index[summoner_team]+1, "/", rival_index+1, ",", jungler_index[summoner_team != 1]+1)

# NaN 값 처리 - NaN 값의 위치에 빈 list를 넣어줍니다.
d = df_kill_events
d.assistingParticipantIds = [[] if x is np.NaN else x for x in d.assistingParticipantIds]   # is를 써야 하는 이유 : NaN 값은 언제나 다른 값이므로 객체를 비교해야합니다.

6 , 7 / 4 , 5


### 3. 원하는 이벤트 데이터를 찾을 수 있는 mask 만들기

In [21]:
# mask 생성
mask_summoner_kill = d.killerId == summoner_index+1
mask_summoner_assist = d.assistingParticipantIds.map(lambda arr : summoner_index+1 in arr)
mask_summoner_jungler_kill = d.killerId == jungler_index[summoner_team]+1
mask_summoner_jungler_assist = d.assistingParticipantIds.map(lambda arr : jungler_index[summoner_team]+1 in arr)
# print(mask_summoner_kill)
# display(d[mask_summoner_jungler_assist & mask_summoner_kill])
# display(d[mask_summoner_jungler_kill & mask_summoner_assist])
# display(d[mask_summoner_kill | mask_summoner_assist])

,assistingParticipantIds,bounty,killStreakLength,killerId,timestamp,type,victimDamageDealt,victimDamageReceived,victimId,position.x,position.y


,assistingParticipantIds,bounty,killStreakLength,killerId,timestamp,type,victimDamageDealt,victimDamageReceived,victimId,position.x,position.y


,assistingParticipantIds,bounty,killStreakLength,killerId,timestamp,type,victimDamageDealt,victimDamageReceived,victimId,position.x,position.y
4,"[6, 7, 8, 9]",180,0,10,279658,CHAMPION_KILL,NaN,"[{'basic': False, 'magicDamage': 116, 'name': ...",2,7120,7658
5,"[6, 7, 8, 10]",180,0,9,280252,CHAMPION_KILL,"[{'basic': False, 'magicDamage': 39, 'name': '...","[{'basic': False, 'magicDamage': 12, 'name': '...",1,6543,7194
6,"[6, 7, 9, 10]",180,0,8,285935,CHAMPION_KILL,"[{'basic': False, 'magicDamage': 100, 'name': ...","[{'basic': False, 'magicDamage': 19, 'name': '...",5,6312,6151
7,"[6, 9, 10]",180,1,8,290890,CHAMPION_KILL,"[{'basic': False, 'magicDamage': 59, 'name': '...","[{'basic': False, 'magicDamage': 15, 'name': '...",3,5570,5289
9,"[6, 7, 8, 10]",180,1,9,319987,CHAMPION_KILL,"[{'basic': False, 'magicDamage': 41, 'name': '...","[{'basic': True, 'magicDamage': 0, 'name': 'HA...",1,5700,5831
13,[6],180,0,9,409966,CHAMPION_KILL,"[{'basic': False, 'magicDamage': 73, 'name': '...","[{'basic': False, 'magicDamage': 119, 'name': ...",4,6830,7009
14,"[6, 7, 8, 10]",180,1,9,444960,CHAMPION_KILL,"[{'basic': False, 'magicDamage': 1, 'name': 'N...","[{'basic': False, 'magicDamage': 92, 'name': '...",1,6661,5671
15,[8],180,0,6,475262,CHAMPION_KILL,NaN,"[{'basic': True, 'magicDamage': 0, 'name': 'Ka...",5,5125,4506
17,"[6, 9, 10]",180,0,7,499673,CHAMPION_KILL,"[{'basic': False, 'magicDamage': 0, 'name': 'L...","[{'basic': False, 'magicDamage': 98, 'name': '...",2,7327,7058
20,"[6, 10]",180,0,8,524356,CHAMPION_KILL,"[{'basic': True, 'magicDamage': 0, 'name': 'Na...","[{'basic': False, 'magicDamage': 164, 'name': ...",1,7689,7406


### 4. 정글러 관여율 구하기

In [17]:
# 정글러 관여율을 [(정글러의 어시, 나의 킬) + (나의 어시, 정글러의 킬)]/(나의 킬 + 나의 어시) 로 가정합니다.
a = len(d[mask_summoner_kill & mask_summoner_jungler_assist]) + len(d[mask_summoner_assist & mask_summoner_jungler_kill])
b = len(d[mask_summoner_assist + mask_summoner_kill])
# print(a, b, a/b)
print("JP-D = %.2f" % (a/b))

JP-D = 0.50


## 모든 경기에 대한 분석으로 확대하기
모든 통계치를 문제없이 구했으므로 모든 경기에 대한 분석으로 확장시켜보겠습니다

In [18]:
# 각 경기로부터의 통계치를 저장할 변수를 선언합니다.
list_csd = []
list_gd = []
list_jp_d = []

# 반복문을 활용하여 모든 경기에 대한 분석을 진행합니다.
for match_index in range(match_length):
    ### 맞라인 상대와 각 팀 정글러의 index 구하기 ###
    summoner_position = data.individualPosition[match_index]    # 소환사의 포지션을 확인합니다.
    if summoner_position == "invalid": continue                 # 소환사의 포지션이 invalid 즉, 다른 모드의 케이스는 무시합니다.

    # 따라서 다른 소환사의 데이터를 다루기 위해 새로운 변수를 선언합니다.
    participant = df_participants.iloc[match_index*10: match_index*10+10].reset_index() # 해당 경기의 소환사들의 데이터를 불러옵니다.

    jungler_index = [-1, -1]    # [블루팀 정글러의 index, 레드팀 정글러의 index]
    for i in range(10):
        # 각 팀 정글러의 index 구하기
        if participant.individualPosition[i] == "JUNGLE":
            if i < 5: jungler_index[0] = i
            else: jungler_index[1] = i
        # 내 index 구하기
        if participant.puuid[i] == summoner_puuid: 
            summoner_index = i
            if summoner_index < 5 : summoner_team = 0   # 0: blue / 1: red
            else: summoner_team = 1                     # 본인 팀에 대한 지표로, 정글러의 index를 불러올 때 사용합니다. 
        # 맞라인 상대의 index 구하기
        elif participant.individualPosition[i] == summoner_position:
            rival = participant.iloc[i]
            rival_index = i

    ### CSD 구하기 ###
    # 'match_index' 번째 경기의 tl_info로부터 't 분' 시점의 '참가자들의 데이터'를 가져옵니다.
    tl_participants_frames          = [tl_info[match_index][t]["participantFrames"] for t in range(len(tl_info[match_index]))]
    if len(tl_participants_frames) <= 15 : continue                 # 15분 전에 끝난 경기는 무시합니다.
    tl_15_participants_frames       = tl_participants_frames[15]    # 15분 데이터
    df_tl_15_participants_frames    = pd.json_normalize([tl_15_participants_frames[str(x+1)] for x in range(10)])

    csd = df_tl_15_participants_frames.minionsKilled[summoner_index] - df_tl_15_participants_frames.minionsKilled[rival_index]
    list_csd.append(csd)

    ### GD 구하기 ###
    gd = df_tl_15_participants_frames.totalGold[summoner_index] - df_tl_15_participants_frames.totalGold[rival_index]
    list_gd.append(gd)

    ### JP-D 구하기 ###
    # 'match_index' 번째 경기의 tl_info로부터 't 분' 시점의 '발생한 이벤트 목록'을 가져옵니다.
    tl_events = [tl_info[match_index][t]["events"] for t in range(len(tl_info[match_index]))]
    tl_kill_events = []
    for i in range(len(tl_events)):
        for event in tl_events[i]:
            if event["type"] == "CHAMPION_KILL":
                tl_kill_events.append(pd.json_normalize(event))
                
    # 배열로부터 생성된 데이터프레임은 index를 다시 설정해주는 것이 좋습니다.
    df_kill_events = pd.concat(tl_kill_events).reset_index()    
    del df_kill_events['index']

    # 주의! 타임라인 데이터에서의 참가자들의 index는 1부터 시작합니다. (앞서 다루었던 index는 전부 0~9 사이의 값)

    # 앞에서 구한 "본인의 index, 본인 팀 정글의 index / 맞라인 상대의 index, 상대 팀 정글의 index"를 출력해보았습니다.
    # print(summoner_index+1, ",", jungler_index[summoner_team]+1, "/", rival_index+1, ",", jungler_index[summoner_team != 1]+1)

    # NaN 값 처리 - NaN 값의 위치에 빈 list를 넣어줍니다.
    d = df_kill_events
    d.assistingParticipantIds = [[] if x is np.NaN else x for x in d.assistingParticipantIds]   
    # is를 써야 하는 이유 : NaN 값은 언제나 다른 값이므로 객체를 비교해야합니다.
    
    # mask 생성
    mask_summoner_kill = d.killerId == summoner_index+1
    mask_summoner_assist = d.assistingParticipantIds.map(lambda arr : summoner_index+1 in arr)
    mask_summoner_jungler_kill = d.killerId == jungler_index[summoner_team]+1
    mask_summoner_jungler_assist = d.assistingParticipantIds.map(lambda arr : jungler_index[summoner_team]+1 in arr)

    # 정글러 관여율을 [(정글러의 어시, 나의 킬) + (나의 어시, 정글러의 킬)]/(나의 킬 + 나의 어시) 로 가정합니다.
    a = len(d[mask_summoner_kill & mask_summoner_jungler_assist]) + len(d[mask_summoner_assist & mask_summoner_jungler_kill])
    b = len(d[mask_summoner_assist + mask_summoner_kill])
    list_jp_d.append(a/b)
    

In [19]:
data_csd = pd.Series(list_csd, name="csd_15")
display(data_csd.head())
data_gd = pd.Series(list_gd, name="gd_15")
display(data_gd.head())
data_jp_d = pd.Series(list_jp_d, name="jp_d")
display(data_jp_d.head())

0    11
1    52
2   -15
3    -3
4     6
Name: csd_15, dtype: int64

0    1321
1    1558
2    -381
3    2085
4     745
Name: gd_15, dtype: int64

0    0.000000
1    0.500000
2    0.285714
3    0.187500
4    0.500000
Name: jp_d, dtype: float64

# Remark: LCK 따라잡기
- KDA
- DPM (분당 데미지)
- DMG% (팀 내 데미지 비율)
- DMG/GOLD (골드 당 데미지)
- CSD@15 (15분 CS 차이)
- GD@15 (15분 골드 차이)
- JP-D (정글러 관여율 차이)

In [20]:
print("""
KDA         : %.2f
DPM         : %.2f
DPM%%        : %.1f%%
DMG/GOLD    : %.1f%%
CSD@15      : %.2f
GD@15       : %.2f
JP-D        : %.1f%%
""" % (
    data_kda.kda.mean(),
    data_dpm.dpm_to_champions.mean(),
    data_dmg_per.mean()*100,
    data_dmg_by_gold.mean()*100,
    data_csd.mean(),
    data_gd.mean(),
    data_jp_d.mean()*100
))


KDA         : 3.53
DPM         : 721.26
DPM%        : 9.5%
DMG/GOLD    : 151.8%
CSD@15      : -4.08
GD@15       : -79.82
JP-D        : 23.1%



# 추가로 고려해볼 것
- key player니까 맞라인 상대가 아니라 팀 내 데이터를 분석한 것일 가능성이 더 높아보임.
    - 다시 해야지... ㅎ